Samuel Karkache
Trey Marcantonio

Differential Power Analysis metrics for Security Analysis of Integrated Circuits MQP. These DPA metric functions will be able to read multiple file types (ChipWhisper, LeCroyDSO, etc...) and perform different metric analysis on the associated power traces. This provides an interface for traces stored in different file formats and allows for efficient processing of large datasets. 

In [57]:
import numpy as np
import chipwhisperer as cw
import h5py as h5
import trsfile as trs
import matplotlib.pyplot as plt
from scipy import stats

# Function:
# get the traces and plaintext list from a given file and convert into two arrays
# Arguments:
# - file_path: the file path of the trace file 
# - file_type: the file type of the trace file, can either be a CW file or a .trs file
# - time_start: the start sample of each trace, will default to 0
# - time_end: the end sample of each trace, will default to the length of the trace
def get_traces_from_file(file_path, file_type, time_start=None, time_end=None) :
    
    print("Attempting to open " + file_path + " as a " + file_type + " file...")

    # ChipWhisperer File Type
    if file_type == "CW" :
        try:
            project = cw.open_project(file_path)
        except OSError:
            print("Unable to find ChipWhisperer file")
            return None, None
            
        print("Total Number of Traces: " + project.traces.seg_len)
        
        trace_list=[]
        plaintext_list=[]
        
        for trace in project.traces:
            trace_list.append(trace.wave[time_start:time_end]) # TODO: This implementation needs to be tested
            plaintext_list.append(trace.textin)
        
        return trace_list, plaintext_list 
        
    # Trs File Tpe    
    elif file_type == ".trs" :

        with trs.open(file_path, 'r') as traces : 
            for header, value in traces.get_headers().items():
                print(header, '=', value)
            print("===========================================")    
            print("Total Number of Traces: " + str(len(traces)))
            
            trace_list=[]
            plaintext_list=[]
            
            for i, trace in enumerate(traces[0:len(traces)]): # enumerate over all traces in file
                # if no time start/end put entire trace into array
                if time_start is None :
                    time_start = 0
                if time_end is None :
                    time_end = len(trace)
                trace_list.append(trace[time_start:time_end]) # read specific time samples
                plaintext_list.append(trace.parameters['INPUT'].value[:])
                                
            return trace_list, plaintext_list
    else :
        print("Unknown File Type")
        return None, None            

In [ ]:
# Signal to Noise Ratio metric 
#   - labels: An array of arrays. Each index of the labels array (i.e. labels[0])
#     is a label group containing the corresponding power traces.
# return: the SNR signal 
def signal_to_noise_ratio(labels):
   # statistical mean and variances of each set 
   set_means = []
   set_variances = []
   
   for label in labels:
       set_means.append(np.mean(label, axis=0)) # take the mean along the column
       set_variances.append(np.var(label, axis=0)) # take the variance along the column
   
   # calculate overall mean and variance
   overall_mean= np.mean(set_means, axis=0)              
   overall_variance = np.var(set_variances, axis=0)

   # perform SNR calculation
   l_d = np.zeros(len(set_means[0]))
   for mean in set_means:
       l_d = np.add(l_d, np.square(np.subtract(mean, overall_mean)))
   l_n = overall_variance

   snr = np.divide(l_d, l_n)

   return snr

In [37]:
# Score metric
#   - traces: The trace set to be evaluated
#   - score_fcn: Function callback that takes two arguments, traces and a guess candidate
#                and returns a "score" such that the higher the value, the more likely the 
#                key candidate is the actual key
#   - partitions: The number of partitions of the key full key.
# return: A 2D array rank. The value rank[i] are the key guess rankings for partition i. 
#         The value of rank[i][0] is the highest ranked key guess for partition i.
def score_and_rank(traces, score_fcn, key_candidates, partitions):
        ranks = []
        # for each key partition        
        for i in range(partitions): 
            dtype = [('key', int), ('score', 'float64')]
            partition_scores = np.array([], dtype=dtype)
            
            # for each key guess in the partition score the value and add to list
            for k in key_candidates:
                score_k = score_fcn(traces, k)
                key_score = np.array([(k, score_k)], dtype=dtype)
                partition_scores = np.append(partition_scores, key_score)
                
            # rank each key where partition_ranks[0] is the key that scored the highest
            partition_ranks = np.array([key_score[0] for key_score in np.sort(partition_scores, order='score')[::-1]])
            
            ranks.append(partition_ranks)
        return ranks

In [ ]:
# T-test with TVlA metric. In general |t| > th where th = 4.5 means that the system is leaky
#   - fixed:  Trace set recorded with a fixed plaintext 
#   - random: Trace set recorded with a random set of plaintexts
# return: t_statistic and p-value
def t_test_tvla(fixed, random):
    
    # determine t_statistic and p-value using scipy 
    t_statistic, p_value = stats.ttest_ind(fixed, random)
    
    # high t-statistic and low p-values indicate that a given time sample leaks information
    return t_statistic, p_value

In [58]:
# Test reading on trs file
traces_list, plaintext = get_traces_from_file("traces1.trs", ".trs", 0, 255)